In [1]:
# Import libraries
import os
import boto3
import sagemaker

from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# initialize preconfigurations
region = boto3.Session().region_name
role = get_execution_role()

In [3]:
print(region)


us-east-1


In [4]:
print(role)


arn:aws:iam::441256151353:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [5]:
# Create S3 bucket

bucket = 'sagemaker-dallas-pca-ml'  #sagemaker.Session().default_bucket()

bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

print(bucket)
print(bucket_path)

sagemaker-dallas-pca-ml
https://s3-us-east-1.amazonaws.com/sagemaker-dallas-pca-ml


In [6]:
model_file_name = "dallas-local-RF-model"

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [8]:
import sklearn
print(sklearn.__version__)

1.3.2


In [9]:
import joblib

mymodel = joblib.load(model_file_name)

In [10]:
#import test file
import numpy as np

file_name = (
    "test_rf_sample.csv"  
)

with open(file_name, "r") as f:
    mypayload = np.loadtxt(f, delimiter=",")
    
print(mypayload)    

mymodel.predict(mypayload)

[[ 7.50010000e+04  5.20105908e-01 -1.65986856e-01  2.01100000e+03
   3.00000000e+00]
 [ 7.50010000e+04  5.20105908e-01 -1.65986856e-01  2.01100000e+03
   4.00000000e+00]
 [ 7.50010000e+04  5.20105908e-01 -1.65986856e-01  2.01200000e+03
   1.00000000e+00]
 [ 7.50010000e+04  5.20105908e-01 -1.65986856e-01  2.01200000e+03
   2.00000000e+00]
 [ 7.50010000e+04  5.20105908e-01 -1.65986856e-01  2.01200000e+03
   3.00000000e+00]]


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([0.05237651, 0.05368977, 0.05592978, 0.05438171, 0.05456732])

In [11]:

# Create a tar.gz model file as this is the format required by Sagemaker for deployment.
joblib.dump(mymodel, model_file_name)

import tarfile

# Creating a tar.gz archive
with tarfile.open(model_file_name + '.tar.gz', 'w:gz') as tar:
    tar.add(model_file_name)

In [12]:
!tar czvf model.tar.gz $model_file_name


dallas-local-RF-model


In [13]:
# Upload the pre-trained model to S3

#### prefix in S3
prefix = "sagemaker/sagemaker-dallas-pca-ml"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

sagemaker/sagemaker-dallas-pca-ml/dallas-local-RF-model/model.tar.gz


In [14]:
## Set up hosting for the model
'''
Import model into hosting
This involves creating a SageMaker model from the model file previously uploaded to S3.
'''

'\nImport model into hosting\nThis involves creating a SageMaker model from the model file previously uploaded to S3.\n'

In [15]:
# Create a Sagemaker model
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [16]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

https://s3-us-east-1.amazonaws.com/sagemaker-dallas-pca-ml/sagemaker/sagemaker-dallas-pca-ml/dallas-local-RF-model/model.tar.gz
arn:aws:sagemaker:us-east-1:441256151353:model/dallas-local-rf-model2024-02-24-22-18-28
CPU times: user 88.7 ms, sys: 12.3 ms, total: 101 ms
Wall time: 764 ms


In [17]:
# Create endpoint configuration
'''
Create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. 
In addition, the endpoint configuration describes the instance type required for model deployment.
'''
from time import gmtime, strftime

endpoint_config_name = "dallas-pca-ml-EndpointConfig" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

dallas-pca-ml-EndpointConfig2024-02-24-22-18-29
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:441256151353:endpoint-config/dallas-pca-ml-endpointconfig2024-02-24-22-18-29


In [18]:
# Create endpoint
'''
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. 
The end result is an endpoint that can be validated and incorporated into production applications.
'''
import time
endpoint_name = "dallas-pca-ml-Endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

dallas-pca-ml-Endpoint-2024-02-24-22-18-29
arn:aws:sagemaker:us-east-1:441256151353:endpoint/dallas-pca-ml-endpoint-2024-02-24-22-18-29
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:441256151353:endpoint/dallas-pca-ml-endpoint-2024-02-24-22-18-29
Status: InService


In [19]:
# Validate the model for use
'''
Now you can obtain the endpoint from the client library using the result from previous operations 
and generate classifications from the model using that endpoint.
'''
runtime_client = boto3.client("runtime.sagemaker")

In [29]:
%%time
import json


file_name = (
    "test_rf_sample.csv"  # customize to your test file
)

with open(file_name, "r") as f:
    payload = f.read().strip()
    
    
print("Payload :\n")

print(payload)
print()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

##print(response)

print("Results :\n")
print()

result = response["Body"].read().decode("utf-8")



Payload :

75001.0,0.5201059082516404,-0.1659868557183671,2011,3
75001.0,0.5201059082516404,-0.1659868557183671,2011,4
75001.0,0.5201059082516404,-0.1659868557183671,2012,1
75001.0,0.5201059082516404,-0.1659868557183671,2012,2
75001.0,0.5201059082516404,-0.1659868557183671,2012,3



ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "Unable to load model: 'numpy.ndarray' object has no attribute 'set_param'". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/dallas-pca-ml-Endpoint-2024-02-24-22-18-29 in account 441256151353 for more information.

In [30]:
# Delete the Endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)
runtime_client.delete_endpoint(EndpointName=endpoint_name)

#dallas-pca-ml-Endpoint-2024-02-24-21-28-54

AttributeError: 'SageMakerRuntime' object has no attribute 'delete_endpoint'